In [1]:
import pandas as pd
import glob
from datetime import datetime, timedelta
import pyodbc

# Load all CSV files from the specified folder into a single DataFrame
path = r'C:\\Users\\pskotte\\Desktop\\New folder'  # Folder path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

# Convert Report Date column to datetime format
frame['Report Date'] = pd.to_datetime(frame['Report Date'])

# Filter out dates from the Report Date column that are older than 30 days
threshold_date = datetime.now() - timedelta(days=30)
filtered_frame = frame[frame['Report Date'] > threshold_date]

# Count transactions
transaction_count = filtered_frame.shape[0]

# Connect to the database
cnxn_str = 'Driver={ODBC Driver 17 for SQL Server};Server=tcp:azwsynt00.sql.azuresynapse.net;Database=AZWSYNT00;Authentication=ActiveDirectoryIntegrated'
cnxn = pyodbc.connect(cnxn_str)
cursor = cnxn.cursor()

cursor.execute("""
SELECT COUNT(*) AS 'Total Transaction Count'
FROM [ref].[DM_Consumer_Account_Payment_Journal] CAPJ
INNER JOIN [ref].[DM_Consumer_Payment_Journal] CPJ ON CAPJ.[ConsumerPaymentJournalID] = CPJ.[ConsumerPaymentJournalID]
WHERE CAST(CAPJ.ConsumerPaymentEnteredDate AS DATE) BETWEEN CAST(GETDATE() - 30 AS DATE) AND CAST(GETDATE() - 1 AS DATE)
    AND CAPJ.BucketTransactionTypeCode IN (
        2
    )
    --DEFINE CARD PAYMENT FIELDS
    AND PaymentMemoID IN (
        23,
        13,
        12
    )
    AND ConsumerAccountPaymentReversingEntryID IS NULL
""")
count_result = cursor.fetchone()[0]  # Fetch the result

# Calculate DM Return Rate as a percentage
dm_return_rate = (transaction_count / count_result)

# Export to a CSV file with DM Return Rate
summary_df = pd.DataFrame({'DM Return Rate': [dm_return_rate]})
summary_df.to_csv(r'C:\\Users\\pskotte\\Desktop\\Power Bi Data\\ACH Return Rate DM.csv', index=False)

C:\Users\pskotte\AppData\Local\Temp\ipykernel_8312\1366101009.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  frame['Report Date'] = pd.to_datetime(frame['Report Date'])
